# Import Statements

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# Mapping packages:
import folium

In [7]:
# import geopandas as gpd

To install geopandas and folium:

`conda install -c conda-forge geopandas`

`conda install -c conda-forge folium `

then:

`jupyter nbextension enable vega --py --sys-prefix`

to enable the necessary notebook extension to use Folium.

# Importing Data

In [49]:
# Don't edit raw dataframes after they have been imported:
train_raw = pd.read_csv('assets/train.csv')
test_raw = pd.read_csv('assets/test.csv', index_col=0)
weather_raw = pd.read_csv('assets/weather.csv')
spray_raw = pd.read_csv('assets/spray.csv')

# These can be the cleaned versions:
train = train_raw.copy().drop(['Address','Block','Street','AddressNumberAndStreet','AddressAccuracy'], axis=1)
test = test_raw.copy().drop(['Address','Block','Street','AddressNumberAndStreet', 'AddressAccuracy'], axis=1)
# If we're using latitude and longitude we can drop out the address info.
weather = weather_raw.copy()
spray = spray_raw.copy()

# Cleaning Data and EDA, train and test

## Simple Map showing the Weather Stations (airports)

In [28]:
m = folium.Map(
    location=[41.883844, -87.632162],
    tiles='openstreetmap',
    zoom_start=10,
    max_zoom = 10,
    min_zoom = 10
)

#folium.tooltip = 'Click Me!' #WARNING: Tooltips are not in this release; use the dev one if you want it

folium.Marker([41.973312, -87.910576], popup='OHare Iternational Airport', icon=folium.Icon(icon='plane', color = 'red')).add_to(m)
folium.Marker([41.786, -87.752], popup='Midway International Airport', icon=folium.Icon(icon='plane', color = 'green')).add_to(m)

m

## Examining the training set:

In [27]:
test['Date'] = pd.to_datetime(test['Date'], format='%Y/%m/%d')
train['Date'] = pd.to_datetime(train['Date'], format='%Y/%m/%d')
train.head()

,Date,Species,Trap,Latitude,Longitude,NumMosquitos,WnvPresent
0,2007-05-29,CULEX PIPIENS/RESTUANS,T002,41.954690,-87.800991,1,0
1,2007-05-29,CULEX RESTUANS,T002,41.954690,-87.800991,1,0
2,2007-05-29,CULEX RESTUANS,T007,41.994991,-87.769279,1,0
3,2007-05-29,CULEX PIPIENS/RESTUANS,T015,41.974089,-87.824812,1,0
4,2007-05-29,CULEX RESTUANS,T015,41.974089,-87.824812,4,0


### Exploring Species Column -- Ben

In [11]:
train['Species'].value_counts() # WNV is transmitted by all mosquitos in the Culex genus.

CULEX PIPIENS/RESTUANS    4752
CULEX RESTUANS            2740
CULEX PIPIENS             2699
CULEX TERRITANS            222
CULEX SALINARIUS            86
CULEX TARSALIS               6
CULEX ERRATICUS              1
Name: Species, dtype: int64

[Which mosquitos spread WNV?](https://wwwnc.cdc.gov/eid/article/7/6/01-0617_article)

In [12]:
# In the training dataset, only Culex Pipiens and Culex Restuans carried WNV.
train.groupby(by ='Species')['NumMosquitos','WnvPresent'].mean()

,NumMosquitos,WnvPresent
Species,,
CULEX ERRATICUS,7.000000,0.000000
CULEX PIPIENS,16.550945,0.088922
CULEX PIPIENS/RESTUANS,13.945286,0.055135
CULEX RESTUANS,8.551460,0.017883
CULEX SALINARIUS,1.686047,0.000000
CULEX TARSALIS,1.166667,0.000000
CULEX TERRITANS,2.297297,0.000000


### Exploring the Trap Column

In [40]:
print(str(train['Trap'].nunique()) + ' traps in the training set \n')
print(str(test['Trap'].nunique()) + ' traps in the training set')

136 traps in the training set 

149 traps in the training set


# Cleaning Data and EDA, weather -- Steve

NB: Weather data is from May 2007 to the end of October 2014

## Checking for Missing Data and Non-numeric Values

In [70]:
# No null data
weather.head(2)

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,...,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,Station Location,Latitude,Longitude
0,1,2007-05-01,83,50,67,14,51,56,0,2,...,0.0,0.00,29.10,29.82,1.7,27,9.2,O'HARE,41.970748,-87.908336
1,2,2007-05-01,84,52,68,M,51,57,0,3,...,M,0.00,29.18,29.82,2.7,25,9.6,MIDWAY,41.787957,-87.752359


In [71]:
weather.dtypes

Station                      int32
Date                datetime64[ns]
Tmax                         int64
Tmin                         int64
Tavg                        object
Depart                      object
DewPoint                     int64
WetBulb                     object
Heat                        object
Cool                        object
Sunrise                     object
Sunset                      object
CodeSum                     object
Depth                       object
Water1                      object
SnowFall                    object
PrecipTotal                 object
StnPressure                 object
SeaLevel                    object
ResultSpeed                float64
ResultDir                    int64
AvgSpeed                    object
Station Location            object
Latitude                   float64
Longitude                  float64
dtype: object

In [74]:
weather['Tavg'].astype(int)

ValueError: invalid literal for int() with base 10: 'M'

## Labeling Weather Stations -- Steve

In [50]:
weather['Date'] = pd.to_datetime(weather['Date'], format='%Y/%m/%d')

In [51]:
weather['Station'] = weather['Station'].astype(str)

#Converting to string so I can run the Map function below.

weather["Station Location"] = weather["Station"].map(lambda x: "O'HARE" if '1' in x else "MIDWAY" if '2' in x else "")

In [52]:
weather['Station'] = weather['Station'].astype(int)
#And back to integer now for math usage

# O'Hare: 41.970748, -87.908336
# Midway: 41.787957, -87.752359

weather["Latitude"] = weather["Station Location"].map(lambda x: "41.970748" if "O'HARE" in x else "41.787957" if "MIDWAY" in x else "")
weather["Longitude"] = weather["Station Location"].map(lambda x: "-87.908336" if "O'HARE" in x else "-87.752359" if "MIDWAY" in x else "")

In [53]:
#Mapping latitude and longitude for the airports

weather["Latitude"] = weather["Latitude"].astype(float)
weather["Longitude"] = weather["Longitude"].astype(float)

#Converting the string entries of location back into floats for math usage

In [54]:
weather.head()

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,...,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,Station Location,Latitude,Longitude
0,1,2007-05-01,83,50,67,14,51,56,0,2,...,0.0,0.00,29.10,29.82,1.7,27,9.2,O'HARE,41.970748,-87.908336
1,2,2007-05-01,84,52,68,M,51,57,0,3,...,M,0.00,29.18,29.82,2.7,25,9.6,MIDWAY,41.787957,-87.752359
2,1,2007-05-02,59,42,51,-3,42,47,14,0,...,0.0,0.00,29.38,30.09,13.0,4,13.4,O'HARE,41.970748,-87.908336
3,2,2007-05-02,60,43,52,M,42,47,13,0,...,M,0.00,29.44,30.08,13.3,2,13.4,MIDWAY,41.787957,-87.752359
4,1,2007-05-03,66,46,56,2,40,48,9,0,...,0.0,0.00,29.39,30.12,11.7,7,11.9,O'HARE,41.970748,-87.908336


In [55]:
weather2011 = weather.loc[(weather['Date'] > '2011-05-01') & (weather['Date'] <= '2011-10-31')]
weather2013 = weather.loc[(weather['Date'] > '2013-05-01') & (weather['Date'] <= '2013-10-31')]

#Making a subset of the weather data over just these years as thats all the spray data we have.

weather2011 = weather2011.reset_index(drop=True)
weather2013 = weather2013.reset_index(drop=True)

# Cleaning Data and EDA, spray

## Steve's work mapping the spray data:

In [22]:
spray.head()

,Date,Time,Latitude,Longitude
0,2011-08-29,6:56:58 PM,42.391623,-88.089163
1,2011-08-29,6:57:08 PM,42.391348,-88.089163
2,2011-08-29,6:57:18 PM,42.391022,-88.089157
3,2011-08-29,6:57:28 PM,42.390637,-88.089158
4,2011-08-29,6:57:38 PM,42.390410,-88.088858


In [27]:
spray['CombinedLoc'] = spray.Latitude.astype(str).str.cat(spray.Longitude.astype(str), sep=', ')

spray.Date.dtype

spray['Date'] = pd.to_datetime(spray['Date'], format='%Y/%m/%d')

#2011: 08-29 to 09/07
#2013: 07-17 to 09/05

spray2011 = spray.loc[(spray['Date'] > '2011-08-29') & (spray['Date'] <= '2011-09-07')]
spray2013 = spray.loc[(spray['Date'] > '2013-07-17') & (spray['Date'] <= '2013-09-05')]

spray2011 = spray2011.reset_index(drop=True)
spray2013 = spray2013.reset_index(drop=True)

In [28]:
spray2011.head()

,Date,Time,Latitude,Longitude,CombinedLoc
0,2011-09-07,7:52:52 PM,41.981433,-87.787777,"41.9814333333, -87.7877766667"
1,2011-09-07,7:53:02 PM,41.980998,-87.787778,"41.9809983333, -87.7877783333"
2,2011-09-07,7:53:12 PM,41.980560,-87.787762,"41.98056, -87.7877616667"
3,2011-09-07,7:53:22 PM,41.980198,-87.787758,"41.9801983333, -87.7877583333"
4,2011-09-07,7:53:32 PM,41.979752,-87.787765,"41.9797516667, -87.787765"


### Spraying locations in 2011:

In [29]:
from folium import plugins
from folium.plugins import MarkerCluster

location2011 = spray2011['Latitude'].mean(), spray2011['Longitude'].mean()
# locationlist2011 = spray2011[["Latitude","Longitude"]].values.tolist()

map2011 = (folium.Map(location=location2011,zoom_start=13))

for index,row in spray2011.iterrows():
    folium.CircleMarker([row['Latitude'], row['Longitude']],
                        radius=3,
                        fill = True,
                        color = "#4286f4", 
                        fill_color="#4286f4"
                       ).add_to(map2011)

map2011.save('2011 Spray.html') 

### Spraying locations in 2013:

In [30]:
map2013 = (folium.Map(location=[41.977049, -87.768738],zoom_start=11))

for index,row2 in spray2013.iterrows():
    folium.CircleMarker([row2['Latitude'], row2['Longitude']],
                       radius=3,
                        fill = True,
                        color = "#b22323", 
                        fill_color="#b22323", # divvy color
                       ).add_to(map2013)

map2013.save('2013 Spray.html')

### All Spraying Locations (2011 & 2013), clustered together

In [31]:
masterspraycluster = (folium.Map(location=[41.843667, -87.803933],zoom_start=10))

marker_cluster = MarkerCluster().add_to(masterspraycluster)

for index,row in spray2011.iterrows():
    folium.CircleMarker([row['Latitude'], row['Longitude']], 
                        radius=3,
                        fill = True,
                        color = "#38ff55", 
                        fill_color="#38ff55").add_to(marker_cluster)
                       

for index, row2 in spray2013.iterrows():
    folium.CircleMarker([row2['Latitude'], row2['Longitude']], 
                        radius=3,
                        fill=True, 
                        color="#ff0000", 
                        fill_color= "#ff0000").add_to(marker_cluster)

masterspraycluster.add_child(folium.LatLngPopup())
masterspraycluster.save('Master Spray (with Clusters).html')

### All Spraying Locations (2011 & 2013), Raw

In [32]:
masterspray = (folium.Map(location=[41.843667, -87.803933],zoom_start=10))

for index,row in spray2011.iterrows():
    folium.CircleMarker([row['Latitude'], row['Longitude']], 
                        radius=3,
                        fill = True,
                        color = "#38ff55", 
                        fill_color="#38ff55").add_to(masterspray)
                       

for index, row2 in spray2013.iterrows():
    folium.CircleMarker([row2['Latitude'], row2['Longitude']], 
                        radius=3,
                        fill=True, 
                        color="#ff0000", 
                        fill_color= "#ff0000").add_to(masterspray)

masterspray.add_child(folium.LatLngPopup())
masterspray.save('Master Spray.html')

In [33]:
# markerspray = (folium.Map(location=[41.843667, -87.803933],zoom_start=10))

# locationlist2011 = spray2011[["Latitude","Longitude"]].values.tolist()
# locationlist2013 = spray2013[["Latitude","Longitude"]].values.tolist()
# icon2011 = folium.Icon(color='red',icon='ok-sign')
# icon2013 = folium.Icon(color='green',icon='ok-sign')

# for index,row in spray2011.iterrows():
#     folium.Marker([row['Latitude'], row['Longitude']], icon=icon2011,
#                   popup=str(row['Date'], 
#                   )).add_to(markerspray)
                       

# for index, row2 in spray2013.iterrows():
#     folium.Marker([row2['Latitude'], row2['Longitude']], icon=icon2013, 
#                   popup=str(row2['Date'] 
#                   )).add_to(markerspray)
    
# markerspray.save('Marker Spray.html')
   

    
#This Currently does not work.  I'll try and figure it out.

# Merging Train Data and Weather Data

In [11]:
test.head()

,Date,Species,Trap,Latitude,Longitude
Id,,,,,
1,2008-06-11,CULEX PIPIENS/RESTUANS,T002,41.95469,-87.800991
2,2008-06-11,CULEX RESTUANS,T002,41.95469,-87.800991
3,2008-06-11,CULEX PIPIENS,T002,41.95469,-87.800991
4,2008-06-11,CULEX SALINARIUS,T002,41.95469,-87.800991
5,2008-06-11,CULEX TERRITANS,T002,41.95469,-87.800991


In [48]:
weather.head()

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,...,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67,14,51,56,0,2,...,,0,M,0.0,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,M,51,57,0,3,...,,M,M,M,0.00,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51,-3,42,47,14,0,...,BR,0,M,0.0,0.00,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52,M,42,47,13,0,...,BR HZ,M,M,M,0.00,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56,2,40,48,9,0,...,,0,M,0.0,0.00,29.39,30.12,11.7,7,11.9
